# 4、 Model I/O之Prompt Template
Prompt Template，通过模板管理大模型的输入

## 4.1 介绍与分类
Prompt Template是LangChain中的一个概念，接收用户输入，返回一个传递给LLM的信息（即提示词prompt）
在应用开发中，固定的提示词限制了模型的灵活性和适用范围。所以，prompt template是一个模板化的字符串，你可以将变量插入到模板中，从而创建出不同的提示。调用时：
- 以字典作为输入，其中每个键代表要填充的提示模板中的变量
- 输出一个PromptValue。这个PromptValue可以传递给LLM或ChatModel，并且还可以转换为字符串或消息列表。

## 4.2 有几种不同类型的提示模板
- PromptTemplate：LLM提示模板，用于生成字符串提示。它使用Python的字符串来模板提示
- ChatPromptTemplate：聊天提示模板，用于组合各种角色的消息模板，传入聊天模型。
- XxxMessagePromptTemplate：消息提示词模板，包括：SystemMessagePromptTemplate、HumanMessagePromptTemplate、AIMessagePromptTemplate、ChatMessagePromptTemplate等
- FewShotPromptTemplate:样本提示词模板，通过示例来教模型如何回答
- PipelinePrompt：管道提示词模板，用于把几个提示词组合在一起使用
- 自定义模板：运行基于其他模板类来制定自己的提示词模板

## 4.3 PromptTemplate

若包含变量：举例如下

In [3]:
from langchain_core.prompts import PromptTemplate

# 1、 创建prompt实例
"""
    参数中必须指明：input_variables 、 template
"""
prompt = PromptTemplate(template="你是一个{role},你的名字叫{name}",
                        input_variables=["role", "name"]
                        )
print(prompt)

# 两种特殊结构的使用（部分提示词与组合提示词的使用）
prompt = prompt.format(role="人工智能专家",name="小智")
print(prompt)


# 变量赋值两种方式：format() invoke()

input_variables=['name', 'role'] input_types={} partial_variables={} template='你是一个{role},你的名字叫{name}'
你是一个人工智能专家,你的名字叫小智


方式2： form_template() : 推荐

In [5]:
from langchain_core.prompts import PromptTemplate

# 1、 创建prompt实例
prompt = PromptTemplate.from_template(template="你是一个{role},你的名字叫{name}")
print(prompt)

# 两种特殊结构的使用（部分提示词与组合提示词的使用）
prompt = prompt.format(role="人工智能专家",name="小智")
print(prompt)



input_variables=['name', 'role'] input_types={} partial_variables={} template='你是一个{role},你的名字叫{name}'
你是一个人工智能专家,你的名字叫小智


如果提示词模板中不包含变量

In [ ]:
from langchain_core.prompts import PromptTemplate

# 略


## 2、两种特殊结构的使用（部分提示词模板的使用、组合提示词的使用）
### 2.1 部分提示词模板的使用（重点）
举例

In [6]:
from langchain_core.prompts import PromptTemplate

# 定义多变量模板
prompt = PromptTemplate.from_template(
    template="请评价{product}的缺点，包括{aspect1}和{aspect2}",
    partial_variables={"aspect1":"电池续航"}
)

# 使用模板生成提示词
prompt = prompt.format(product="智能手机",aspect2="拍照质量")
print(prompt)

请评价智能手机的缺点，包括电池续航和拍照质量


## 3、给变量赋值的两种方式：format() / invoke()
format(): 参数部分：变量赋值；返回值：str类型

invoke():参数部分：字典；  返回值：promptValue类型

## 4、结合大模型使用

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import os
import dotenv

dotenv.load_dotenv()    # 加载当前目录下的 .env 文件
os.environ.setdefault("OPENAI_API_KEY", os.getenv("OPENAI_API_KEY"))
os.environ.setdefault("OPENAI_BASE_URL", os.getenv("OPENAI_BASE_URL"))

# 获取对话模型
model = ChatOpenAI(
    model="deepseek-chat",
    temperature=0.7,
    max_tokens=20
)


# 定义多变量模板
prompt = PromptTemplate.from_template(
    template="请评价{product}的缺点，包括{aspect1}和{aspect2}",
    partial_variables={"aspect1":"电池续航"}
)

# 使用模板生成提示词
prompt = prompt.format(product="智能手机",aspect2="拍照质量")

response = model.invoke(prompt)
print(response)

E:\shangdj\python\env\conda\envs\myPython3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


请评价智能手机的缺点，包括电池续航和拍照质量
content='智能手机确实带来了巨大便利，但也存在一些普遍缺点，以下是针对电池续航和拍照质量的客观分析，并补充其他常见问题：\n\n---\n\n### **一、电池续航问题**\n1. **续航焦虑普遍存在**  \n   - 高性能处理器、高刷新率屏幕和5G网络大幅增加耗电量，重度用户常需每日多次充电。\n   - 电池容量受机身轻薄化设计限制，技术突破缓慢（如石墨烯电池尚未普及）。\n2. **电池老化速度快**  \n   - 锂离子电池循环寿命有限（通常500-800次循环后容量显著下降），1-2年后续航可能明显缩短。\n3. **快充与无线充的局限性**  \n   - 快充虽普及，但长期高功率充电可能加速电池损耗。\n   - 无线充电效率低、发热高，且需特定场景支持。\n\n---\n\n### **二、拍照质量的局限性**\n1. **硬件依赖与同质化**  \n   - 多数手机依赖算法优化，传感器尺寸、镜头光学素质受机身空间限制，与专业相机差距明显。\n   - 厂商过度追求高像素营销，但像素并非画质唯一标准，暗光环境下噪点、涂抹感问题突出。\n2. **算法优化的副作用**  \n   - 自动美颜、HDR合成可能导致照片失真（如过度锐化、色彩失真），失去真实感。\n   - 多摄像头协同仍存在色彩不一致、变焦跳档等问题。\n3. **专业功能缺失**  \n   - 手动控制选项有限（如光圈、快门速度），RAW格式支持不完善，后期处理空间较小。\n\n---\n\n### **三、其他常见缺点**\n1. **系统碎片化与维护周期短**  \n   - 安卓机型系统更新支持时间短（通常2-3年），旧机型易卡顿且安全更新滞后。\n   - 预装软件和广告推送影响体验。\n2. **维修成本与环保问题**  \n   - 模块化设计不足，屏幕、电池更换费用高，维修难度大。\n   - 快速迭代导致电子垃圾激增，回收体系不完善。\n3. **隐私与安全风险**  \n   - 过度权限申请、数据收集行为常见，敏感信息可能被滥用。\n   - 恶意软件和网络攻击威胁持续存在。\n\n---\n\n### **四、行业改进趋势**\n- **电池技术**：固态电池、硅负极材料等研发中，快充与续航平衡逐